<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/xyz_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-05-29 11:42:40--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8100 (7.9K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   7.91K  --.-KB/s    in 0s      

2022-05-29 11:42:40 (103 MB/s) - ‘utils.py’ saved [8100/8100]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import ModelCreator, get_train_val_ds, ErrorAnalyzer, get_class_weight

     |████████████████████████████████| 25.9 MB 1.5 MB/s 


In [4]:
BATCH_SIZE = 32
IMG_SIZE = (512, 512)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x3/1'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [7]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE)

Found 16580 files belonging to 16 classes.
Found 3508 files belonging to 16 classes.


In [8]:
classes, class_weight = get_class_weight()

In [9]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [10]:
xyz_creator = ModelCreator(HUB_URL, 'xyz-model')

In [11]:
xyz_model = xyz_creator.make_model()
callbacks = xyz_creator.get_callbacks()

In [12]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=15,
                        class_weight=class_weight)

Epoch 1/15
519/519 [==============================] - 2625s 5s/step - loss: 257.3902 - precision: 0.6243 - recall: 0.6216 - auc: 0.5162 - accuracy: 0.6223
Epoch 2/15
519/519 [==============================] - 2435s 5s/step - loss: 151.5698 - precision: 0.7233 - recall: 0.7219 - auc: 0.6255 - accuracy: 0.7222
Epoch 3/15
519/519 [==============================] - 2433s 5s/step - loss: 138.6640 - precision: 0.7474 - recall: 0.7465 - auc: 0.6541 - accuracy: 0.7469
Epoch 4/15
519/519 [==============================] - 2434s 5s/step - loss: 128.2173 - precision: 0.7694 - recall: 0.7683 - auc: 0.6838 - accuracy: 0.7689
Epoch 5/15
519/519 [==============================] - 2433s 5s/step - loss: 100.3120 - precision: 0.7903 - recall: 0.7893 - auc: 0.7120 - accuracy: 0.7896
Epoch 6/15
519/519 [==============================] - 2430s 5s/step - loss: 88.9676 - precision: 0.8045 - recall: 0.8042 - auc: 0.7349 - accuracy: 0.8043
Epoch 7/15
519/519 [==============================] - 2433s 5s/step - l

KeyboardInterrupt: ignored